In [10]:


# ..............................................PART 1 FACE RECOGNITION...............

import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names




In [11]:
# import cv2
# import time
# import mysql.connector
# from simple_facerec import SimpleFacerec

# # MySQL database connection parameters
# db_config = {
#     'host': 'localhost',
#     'user': 'root',
#     'password': '',
#     'database': 'face_recognition'
# }

# # Establish MySQL connection
# conn = mysql.connector.connect(**db_config)
# cursor = conn.cursor()

# # Encode faces from a folder
# sfr = SimpleFacerec()
# sfr.load_encoding_images("images/")

# # Load Camera
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()

#     # Detect Faces
#     face_locations, face_names = sfr.detect_known_faces(frame)
#     for face_loc, name in zip(face_locations, face_names):
#         y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

#         # Get current time
#         current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
#         print(f"Detected Face {face_loc} {name} at {current_time}")

#         # Display face name
#         cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)

#         # Display detection time below the face name
#         cv2.putText(frame, f"{current_time}", (x1, y2 + 50), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 200), 1)

#         # Draw rectangle around the face
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

#         # Store information in MySQL database
#         sql = "INSERT INTO face_detection (name, x1, y1, x2, y2, detection_time) VALUES (%s, %s, %s, %s, %s, %s)"
#         # values = (name, x1, y1, x2, y2, current_time)
#         values = (name, int(x1), int(y1), int(x2), int(y2), current_time)  # Explicitly convert to int
#         cursor.execute(sql, values)
#         conn.commit()

#     cv2.imshow("Frame", frame)

#     key = cv2.waitKey(2)
#     if key == ord('q') or key == 27:  # 'q' key or Esc key
#         break

# # Close MySQL connection
# cursor.close()
# conn.close()

# cap.release()
# cv2.destroyAllWindows()


In [12]:
import cv2
import time
import mysql.connector
import base64
from simple_facerec import SimpleFacerec

# MySQL database connection parameters
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'face_recognition'
}

# Establish MySQL connection
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create face_detection table with an additional column 'face_image' of type BLOB
create_table_query = """
CREATE TABLE IF NOT EXISTS face_detection (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    x1 INT,
    y1 INT,
    x2 INT,
    y2 INT,
    detection_time DATETIME,
    face_image MEDIUMBLOB
);
"""
cursor.execute(create_table_query)
conn.commit()

# Encode faces from a folder
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Load Camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Detect Faces
    face_locations, face_names = sfr.detect_known_faces(frame)
    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        # Get current time
        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(f"Detected Face {face_loc} {name} at {current_time}")

        # Display face name
        cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)

        # Display detection time below the face name
        cv2.putText(frame, f"{current_time}", (x1, y2 + 50), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 200), 1)

        # Draw rectangle around the face
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

        # Capture the face region
        face_image = frame[y1:y2, x1:x2]

        # Convert the face image to a base64-encoded string
        _, img_encoded = cv2.imencode('.png', face_image)
        img_base64 = base64.b64encode(img_encoded.tobytes())

        try:
            # Store information in MySQL database
            sql = """
            INSERT INTO face_detection1 (name, x1, y1, x2, y2, detection_time, face_image) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            values = (name, int(x1), int(y1), int(x2), int(y2), current_time, img_base64)
            cursor.execute(sql, values)
            conn.commit()
        except Exception as e:
            print(f"Error inserting into the database: {e}")

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(2)
    if key == ord('q') or key == 27:  # 'q' key or Esc key
        break

# Close MySQL connection
cursor.close()
conn.close()

cap.release()
cv2.destroyAllWindows()


6 encoding images found.
Encoding images loaded
Detected Face [280 432 452 256] NTAMBARA Etienne at 2024-01-16 02:09:21
Error inserting into the database: 1054 (42S22): Unknown column 'face_image' in 'field list'
Detected Face [280 432 452 256] NTAMBARA Etienne at 2024-01-16 02:09:22
Error inserting into the database: 1054 (42S22): Unknown column 'face_image' in 'field list'
Detected Face [268 448 472 240] NTAMBARA Etienne at 2024-01-16 02:09:23
Error inserting into the database: 1054 (42S22): Unknown column 'face_image' in 'field list'
Detected Face [268 448 472 240] NTAMBARA Etienne at 2024-01-16 02:09:24
Error inserting into the database: 1054 (42S22): Unknown column 'face_image' in 'field list'
Detected Face [244 424 452 220] NTAMBARA Etienne at 2024-01-16 02:09:33
Error inserting into the database: 1054 (42S22): Unknown column 'face_image' in 'field list'
Detected Face [268 424 472 220] NTAMBARA Etienne at 2024-01-16 02:09:42
Error inserting into the database: 1054 (42S22): Unknow

In [13]:
# !pip install pymysql

source codes 
https://pysource.com/2021/08/16/face-recognition-in-real-time-with-opencv-and-python/


https://youtu.be/5yPeKQzCPdI?si=ftccxdgeLwqa8e8N